# 정확도
- 정확도(Accuracy) : 예측결과가 동일한 데이터 건수 / 전체 예측 데이터 건수
- 직관적인 평가지표
- 이진분류의 경우 데이터 구성에 따라서 ML모델의 성능을 왜곡할 수 있음, 정확도만으로 평가하지 말 것
- 불균형한 레이블 값 분포에서 ML모델의 성능을 판단할 경우는 절대 사용하지 않는다.

In [2]:
import numpy as np
import pandas as pd
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
# 그래프 한글 처리하는 코드
import platform
from scipy.stats import *
from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':  # 맥OS 
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows': 
    # 윈도우   
    path = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system...  sorry~~~')

In [3]:
# 타이타닉 데이터 셋을 이용한 분류 및 예측# 타이타닉 데이터 셋을 이용한 분류 및 예측

In [4]:
### fit() 메서드는 아무 것도 수행하지 않고, predict()는 Sex 피처가 1이면 0, 그렇지 않으면 1로 예측하는 단순한 분류기 생성
from sklearn.base import BaseEstimator

class MyDummyClassifier(BaseEstimator):
    # fit 메서드는 아무것도 학습하지 않음
    def fit(self, X, y=None):
        pass
    # predict 메서드는 단순히 Sex 피처가 1이면 0, 아니면 1로 예측
    def predict(self, X):
        pred = np.zeros( (X.shape[0],1) )
        for i in range(X.shape[0]):
            if X['Sex'].iloc[i] == 1:
                pred[i] = 0
            else :
                pred[i] = 1 
        return pred

In [5]:
## 생성된 MyDummyClassifier를 이용해 타이타닉 생존자 예측 수행

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

## Null 처리 함수
def fillna(df):
    df['Age'].fillna(df['Age'].mean(), inplace=True)
    df['Cabin'].fillna('N', inplace=True)
    df['Embarked'].fillna('N', inplace=True)
    df['Fare'].fillna(0, inplace=True)
    return df

## 머신러닝에 불필요한 피처 제거
def drop_features(df):
    df.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)
    return df

## Label Encoding 수행
def format_features(df):
    df['Cabin'] = df['Cabin'].str[:1]
    features = ['Cabin', 'Sex', 'Embarked']
    for feature in features:
        le = LabelEncoder()
        le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

## 앞에서 실행한 Data Preprocessing 함수 호출
def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df

In [6]:
t_df = pd.read_csv('../Data/titanic/train.csv')
y_t_df = t_df['Survived']
x_t_df = t_df.drop('Survived',axis = 1)
x_t_df =  transform_features(x_t_df)

X_train, X_test, y_train, y_test = train_test_split(x_t_df, y_t_df, test_size=0.2, random_state=0)

In [7]:
myclf = MyDummyClassifier()
myclf.fit(X_train,y_train)

mypred = myclf.predict(X_test)
print('Dummy Classifier의 정확도는 : {0:.4f}'.format(accuracy_score(y_test,mypred)))

Dummy Classifier의 정확도는 : 0.7877


In [8]:
# Minist 데이터 셋을 이진 분류 데이터 셋으로 변환 후 정확도 확인 - 레이블 분포가 편중되었을 때

In [9]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score

import numpy as np
import pandas as pd


In [10]:
class MyFakeClassifier(BaseEstimator) :
    def fit(self,X,y) :
        pass
    # 입력값으로 들어오는 X데이터셋의 크기만큼 모두 0값으로 만들어서 반환
    # 즉, 어떤 값이 들어오던 간에 전부 0으로 반환
    def predict(self,X) :
        return np.zeros((len(X),1), dtype=bool)

In [11]:
# 사이킷런 내장 데이터셋인 load_digits() MNIST 데이터 로딩
digits = load_digits()
print(digits.data)
print('digit.data.shape : ', digits.data.shape)
print(digits.target)
print('digits.target.shape : ', digits.target.shape)

[[ 0.  0.  5. ...  0.  0.  0.]
 [ 0.  0.  0. ... 10.  0.  0.]
 [ 0.  0.  0. ... 16.  9.  0.]
 ...
 [ 0.  0.  1. ...  6.  0.  0.]
 [ 0.  0.  2. ... 12.  0.  0.]
 [ 0.  0. 10. ... 12.  1.  0.]]
digit.data.shape :  (1797, 64)
[0 1 2 ... 8 9 8]
digits.target.shape :  (1797,)


In [12]:
# 타겟변수의 값이 7인 경우 1 그 나머지는 전부 0으로 타겟 변수를 변환
y=(digits.target==7).astype(int)
y[:20]

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0])

In [13]:
# 훈련셋, 테스트셋으로 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(digits.data, y, random_state=11)

In [14]:
# 불균형한 레이블 데이터 분포도 확인
print('레이블 테스트 세트 크기 : ', y_test.shape)
print('테스트 세트 레이블 0과 1의 분포도')
print(pd.Series(y_test).value_counts())

레이블 테스트 세트 크기 :  (450,)
테스트 세트 레이블 0과 1의 분포도
0    405
1     45
dtype: int64


In [15]:
# FakeClassifier로 학습/예측 정확도 평가
fakeclf = MyFakeClassifier()
fakeclf.fit(X_train,y_train)
fpred = fakeclf.predict(X_test)
print('모든 예측을 0으로 한 정확도 : {:.3f}'.format(accuracy_score(y_test,fpred)))

모든 예측을 0으로 한 정확도 : 0.900


# Compusion Matrix

In [18]:
from sklearn.metrics import confusion_matrix

# confusion matrix(실제값, 예측값)
confusion_matrix(y_test,fpred) # 예측을 안한 것과 다름 없음

array([[405,   0],
       [ 45,   0]], dtype=int64)

# 정밀도(Precision)와 재현율(Recall)

In [19]:
from sklearn.metrics import precision_score, recall_score
print('정밀도 : ', precision_score(y_test,fpred))
print('재현도 : ', recall_score(y_test,fpred))

정밀도 :  0.0
재현도 :  0.0


C:\Users\choi yeseul\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
## 오차행렬, 정확도, 정밀도, 재현율을 한꺼번에 계산하는 함수 생성
def get_clf_eval(y_test,pred) :
    confusion = confusion_matrix(y_test,pred)
    accuracy = accuracy_score(y_test,pred)
    precision = precision_score(y_test,pred)
    recall = recall_score(y_test,pred)
    print('오차행렬')
    print(confusion)
    print('정확도 : {0:.4f}, 정밀도 : {1:.4f}, 재현율 : {2:.4f}'.format(accuracy,precision,recall))

In [22]:
t_df = pd.read_csv('../Data/titanic/train.csv')
y_t_df = t_df['Survived']
x_t_df = t_df.drop('Survived',axis = 1)
x_t_df =  transform_features(x_t_df)

X_train, X_test, y_train, y_test = train_test_split(x_t_df, y_t_df, test_size=0.2, random_state=0)

In [23]:
from sklearn.linear_model import LogisticRegression
lr_clf = LogisticRegression()

In [24]:
lr_clf.fit(X_train,y_train)

C:\Users\choi yeseul\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [25]:
pred = lr_clf.predict(X_test)

In [26]:
get_clf_eval(y_test,pred)

오차행렬
[[92 18]
 [16 53]]
정확도 : 0.8101, 정밀도 : 0.7465, 재현율 : 0.7681


# Precision/Recall Trade off
- predict_proba() 함수 : 이진분류인 경우 0일 때 확률과 1일 때 확률 확인

In [30]:
pred.reshape(-1,1) # 1차원을 2차원으로 만드는 함수

array([[0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
    

In [28]:
pred_proba = lr_clf.predict_proba(X_test)

In [29]:
pred_proba[:3]

array([[0.85127894, 0.14872106],
       [0.89228674, 0.10771326],
       [0.93359204, 0.06640796]])

In [31]:
# 예측확률 array와 결과값 array를 concat해서 같이 확인
pred_proba_result = np.concatenate([pred_proba,pred.reshape(-1,1)],axis=1)

In [33]:
print('두 개의 클래스 중에서 더 큰 확률을 클래스 값으로 예측 \n',pred_proba_result[:7])

두 개의 클래스 중에서 더 큰 확률을 클래스 값으로 예측 
 [[0.85127894 0.14872106 0.        ]
 [0.89228674 0.10771326 0.        ]
 [0.93359204 0.06640796 0.        ]
 [0.05358208 0.94641792 1.        ]
 [0.31335528 0.68664472 1.        ]
 [0.49006614 0.50993386 1.        ]
 [0.0821846  0.9178154  1.        ]]


In [ ]:
# Binarizer 활용inarizer 활용

In [35]:
from sklearn.preprocessing import Binarizer
X = [[1,-1,2],[2,0,0],[0,1.1,1.2]]
# threshold 기준 값보다 같거나 작으면 0, 크면 1을 반환
binarize = Binarizer(threshold=1.1)
print(binarize.fit_transform(X))

[[0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]]


In [43]:
# 분류 결정 임계값 0.5기반에서 Binarizer를 이용하여 예측값 변환
from sklearn.preprocessing import Binarizer

# 분류 결정 임계값 설정변수
costom_th = 0.5

pred_proba_1 = pred_proba[:,1].reshape(-1,1)
pred_proba_1[:5]

# Binarizer 객체 설정
binarize = Binarizer(threshold=costom_th).fit(pred_proba_1)
custom_predict = binarize.transform(pred_proba_1)
custom_predict[:10]

get_clf_eval(y_test,custom_predict)

오차행렬
[[92 18]
 [16 53]]
정확도 : 0.8101, 정밀도 : 0.7465, 재현율 : 0.7681


In [44]:
# 분류 결정 임계값 설정변수
costom_th = 0.4

pred_proba_1 = pred_proba[:,1].reshape(-1,1)
pred_proba_1[:5]

# Binarizer 객체 설정
binarize = Binarizer(threshold=costom_th).fit(pred_proba_1)
custom_predict = binarize.transform(pred_proba_1)
custom_predict[:10]

get_clf_eval(y_test,custom_predict)
# threshold를 낮추면 재현율이 올라감 / threshold를 높이면 정밀도가 올라감

오차행렬
[[86 24]
 [13 56]]
정확도 : 0.7933, 정밀도 : 0.7000, 재현율 : 0.8116


In [47]:
# 여러개의 분류 결정 임계값을 변경하면서 Binarizer를 이용하여 예측값 반환

# 테스트를 수행할 임계값 리스트
th =  [0.4, 0.45, 0.5, 0.55, 0.6]
def get_eval_by_threshold(y_test,pred_proba_1,th) :
    for custom_th in th :
        binarize = Binarizer(threshold=costom_th).fit(pred_proba_1)
        custom_predict = binarize.transform(pred_proba_1)
        get_clf_eval(y_test,custom_predict)
        
get_eval_by_threshold(y_test,pred_proba[:,1].reshape(-1,1), th)

오차행렬
[[86 24]
 [13 56]]
정확도 : 0.7933, 정밀도 : 0.7000, 재현율 : 0.8116
오차행렬
[[86 24]
 [13 56]]
정확도 : 0.7933, 정밀도 : 0.7000, 재현율 : 0.8116
오차행렬
[[86 24]
 [13 56]]
정확도 : 0.7933, 정밀도 : 0.7000, 재현율 : 0.8116
오차행렬
[[86 24]
 [13 56]]
정확도 : 0.7933, 정밀도 : 0.7000, 재현율 : 0.8116
오차행렬
[[86 24]
 [13 56]]
정확도 : 0.7933, 정밀도 : 0.7000, 재현율 : 0.8116
